In [1]:
#Import library
import numpy as np
import pandas as pd
import vectorbt as bt
import yfinance as yf
import pandas_ta as ta

In [3]:
#Get Data
btc = yf.download('BTC-USD', start='2024-10-20', end='2025-01-01', interval='1h')
print(btc.head())
btc.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
btc = btc.asfreq('h')
btc.tail()

[*********************100%***********************]  1 of 1 completed

Price                             Close          High           Low  \
Ticker                          BTC-USD       BTC-USD       BTC-USD   
Datetime                                                              
2024-10-20 00:00:00+00:00  68229.203125  68418.773438  68229.203125   
2024-10-20 01:00:00+00:00  68150.835938  68233.750000  68147.601562   
2024-10-20 02:00:00+00:00  68161.031250  68202.617188  68133.117188   
2024-10-20 03:00:00+00:00  68178.179688  68183.195312  68113.890625   
2024-10-20 04:00:00+00:00  68223.242188  68288.992188  68169.851562   

Price                              Open  Volume  
Ticker                          BTC-USD BTC-USD  
Datetime                                         
2024-10-20 00:00:00+00:00  68366.562500       0  
2024-10-20 01:00:00+00:00  68222.632812       0  
2024-10-20 02:00:00+00:00  68156.867188       0  
2024-10-20 03:00:00+00:00  68163.093750       0  
2024-10-20 04:00:00+00:00  68171.140625       0  


ValueError: Length mismatch: Expected axis has 5 elements, new values have 6 elements

In [10]:
# Initial parameters
initial_capital = 100000
risk_per_trade = 0.02  # 2%sl_trail=True
stop_loss = 0.18  # 18%
take_profit = 0.40  # 40%

In [11]:
#Technical Indicators
btc['rsi']= ta.rsi(btc['Close'], length=14)
btc['ema_short'] = ta.ema(btc['Close'], length=20)
btc['ema_long'] = ta.ema(btc['Close'], length=50)
btc['vwap'] = ta.vwap(btc['High'], btc['Low'], btc['Close'], btc['Volume'])
bb = ta.bbands(btc['Close'], length=20, std=2)
adx_results = ta.adx(btc['High'], btc['Low'], btc['Close'], length=5)

# Assign to DataFrame
btc['adx'] = adx_results['ADX_5']  # ADX values
btc['a'] = adx_results['DMP_5']    # Positive Directional Movement
btc['b'] = adx_results['DMN_5']    # Negative Directional Movement

btc['bbl'] = bb['BBL_20_2.0']
btc['bbm'] = bb['BBM_20_2.0']
btc['bbu'] = bb['BBU_20_2.0']





/home/bhanu430/.local/lib/python3.12/site-packages/pandas_ta/overlap/vwap.py:23: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  vwap  = wp.groupby(wp.index.to_period(anchor)).cumsum()
/home/bhanu430/.local/lib/python3.12/site-packages/pandas_ta/overlap/vwap.py:24: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  vwap /= volume.groupby(volume.index.to_period(anchor)).cumsum()


In [12]:
#Entry and Exit 
# btc['long_entry'] = ((btc['a'] > btc['b']) & (btc['rsi']<30)).astype(int)
btc['long_entry'] = ((btc['ema_short'] > btc['ema_long']) & (btc['rsi']<30)).astype(int)

# btc['long_exit'] = ((btc['Close']>btc['bbu'])).astype(int)

btc['short_entry'] = ((btc['ema_short'] < btc['ema_long']) & (btc['rsi']>70)).astype(int)
# btc['short_exit'] = (btc['rsi']<30).astype(int)

In [13]:
#Portfolio Construction
pf = bt.Portfolio.from_signals(btc['Close'], entries=btc['long_entry'], short_entries=btc['short_entry'],fees=0.001, slippage=0.001, sl_stop=0.04, sl_trail=True, upon_stop_exit='reverse')

In [14]:
#Returns
returns = pf.total_return()
print(returns)

0.0644828023338988


In [15]:
#Stats
pf.stats()

Start                         2024-10-20 00:00:00+00:00
End                           2024-12-31 23:00:00+00:00
Period                                 73 days 00:00:00
Start Value                                       100.0
End Value                                     106.44828
Total Return [%]                                6.44828
Benchmark Return [%]                          36.980087
Max Gross Exposure [%]                            100.0
Total Fees Paid                                4.663518
Max Drawdown [%]                              15.771699
Max Drawdown Duration                  27 days 12:00:00
Total Trades                                         21
Total Closed Trades                                  20
Total Open Trades                                     1
Open Trade PnL                                -2.500883
Win Rate [%]                                       40.0
Best Trade [%]                                26.531703
Worst Trade [%]                               -3

In [16]:
#Plots
pf.plot(
    subplots=['trades', 'orders', 'trade_pnl', 'cum_returns']
    )


FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': '6a75f805-83f5-429a-b26b-5c3ea7c6b3f1',
              'x': array([datetime.datetime(2024, 10, 20, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 20, 1, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 10, 20, 2, 0, tzinfo=datetime.timezone.utc),
                          ...,
                          datetime.datetime(2024, 12, 31, 21, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 31, 22, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 31, 23, 0, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'xaxis': 'x',
              'y': array([68229.203125 , 68150.8359375, 68161.03125  , ...,